In [1]:
#
### Input File in folder :  csv_data_path
#
source_raw_csv_data_01_NBS_Portfolio_Bristol_230216="data/input/01_NBS_Portfolio_Bristol_230216/01_NBS_Portfolio_Bristol_230216.csv"
#
source_raw_csv_data_01_NBS_Portfolio_Bristol_230216_TWINN="data/input/01_NBS_Portfolio_Bristol_230216/02_NBS_Portfolio_Bristol_230216_TWINN.csv"
#
staged_csv_data_path="data/output/NBS_Portfolio_Bristol_230216_csv/"
#
staged_json_data_path="data/output/NBS_Portfolio_Bristol_230216_json/"
#
##############################Execution##########################
import findspark
findspark.init()
#
import pyspark
from pyspark.sql import functions as pfunc
from pyspark.sql import SQLContext
from pyspark.sql import Window, types
import re
import pandas as pd
import numpy as np
from pandas import DataFrame
from pyspark.sql.types import IntegerType
from pyspark.sql.types import FloatType
from pyspark.sql.functions import udf
from pyspark.sql.functions import *
from scipy.stats import kstest
from scipy import stats
#
sc = pyspark.SparkContext(appName="Airbus NationWide Data - Join and convert CSV files to single JSON by ID key")
sc.setLocalProperty( "spark.sql.crossJoin.enabled" , "true" )
sqlContext = SQLContext(sc)
#
# Join with Internal Curation Data in urltopredict staged folder
from pyspark.sql import functions as F


In [2]:
### remove NULL ID
internaldata_df1=sqlContext.read.csv(source_raw_csv_data_01_NBS_Portfolio_Bristol_230216_TWINN, header=True).persist(pyspark.StorageLevel.MEMORY_AND_DISK_2)
#
df1=internaldata_df1.filter("ID IS NOT NULL").persist(pyspark.StorageLevel.MEMORY_AND_DISK_2)
#
df1.show(1,0)
df1.printSchema()
df1.count()
##

+---+----+---------------+---------------+---------------+---------------+-------------+-------------+-------------+--------------------+------------------+----------------------+----------------------+----------------------+----------------------+--------------------+--------------------+--------------------+----------------------+----------------------+----------------------+----------------------+--------------------+--------------------+--------------------+----------------------+----------------------+----------------------+----------------------+--------------------+--------------------+--------------------+
|ID |UPRN|PRES_UNDEF_F_RR|PRES_UNDEF_P_RR|PRES_UNDEF_T_RR|PRES_UNDEF_C_RR|PRES_DEF_F_RR|PRES_DEF_T_RR|PRES_DEF_C_RR|PRES_FLUVIAL_DEF_BIN|PRES_TIDAL_DEF_BIN|RCP26_2050S_UNDEF_F_RR|RCP26_2050S_UNDEF_P_RR|RCP26_2050S_UNDEF_T_RR|RCP26_2050S_UNDEF_C_RR|RCP26_2050S_DEF_F_RR|RCP26_2050S_DEF_T_RR|RCP26_2050S_DEF_C_RR|RCP45_2050S_UNDEF_F_RR|RCP45_2050S_UNDEF_P_RR|RCP45_2050S_UNDEF_T_R

18085

In [3]:
### remove NULL ID
internaldata_df2=sqlContext.read.csv(source_raw_csv_data_01_NBS_Portfolio_Bristol_230216, header=True).persist(pyspark.StorageLevel.MEMORY_AND_DISK_2)
#
df2=internaldata_df2.filter("ID IS NOT NULL").persist(pyspark.StorageLevel.MEMORY_AND_DISK_2)
#
df2.show(1,0)
df2.printSchema()
df2.count()
##

+---+----+-------------+---------+---------+----------+--------------+----------+--------------+-----------+---------------+-----------+-------------+-----------------+---------------+--------------+---------------+--------------+---------------+--------------+------------------+-----------------+---------------+--------------+---------------+--------------+---------------+--------------+------------------+-----------------+---------------+--------------+---------------+--------------+---------------+--------------+------------------+-----------------+---------------+-----------------------+-------------------+---------------+-------------+-------+-------+--------+--------+--------+--------+--------+-------------+-------------+-------------+-------------+--------------+----------------+----------------+-------------+--------+----------+----------------+---------+---------------+----------------------------------+-------------------------+---------------------+-----------------------+--

18085

In [4]:
# Join DFs by ID
#
join_df3=df2.join(df1, df1['ID']==df2['ID'] , "leftouter")\
.drop(df1['ID']).drop(df1['UPRN'])

#join_df3.createOrReplaceTempView("NBS_Portfolio_Bristol_230216_JOIN")
#sql_join_df3 = sqlContext.sql("select * from NBS_Portfolio_Bristol_230216_JOIN where ID IS NOT NULL and UPRN IS NOT NULL")


In [5]:
#
join_df3.show(1,0)
join_df3.printSchema()
join_df3.count()
##

+---+----+-------------+---------+---------+----------+--------------+----------+--------------+-----------+---------------+-----------+-------------+-----------------+---------------+--------------+---------------+--------------+---------------+--------------+------------------+-----------------+---------------+--------------+---------------+--------------+---------------+--------------+------------------+-----------------+---------------+--------------+---------------+--------------+---------------+--------------+------------------+-----------------+---------------+-----------------------+-------------------+---------------+-------------+-------+-------+--------+--------+--------+--------+--------+-------------+-------------+-------------+-------------+--------------+----------------+----------------+-------------+--------+----------+----------------+---------+---------------+----------------------------------+-------------------------+---------------------+-----------------------+--

18085

In [6]:
join_df3.coalesce(1).write.option("header","true").mode("overwrite").format("csv").save(staged_csv_data_path)
##
print("Conversion of CSV source files join by ID to CSV done!")
####

Conversion of CSV source files join by ID to CSV done!


In [7]:
join_df3.coalesce(1).write.mode("overwrite").format("json").save(staged_json_data_path)
##
print("Conversion of CSV source files join by ID to JSON done!")
####

Conversion of CSV source files join by ID to JSON done!


In [8]:
sc.stop()